# 2023-01-19 13:30
민규사원님이 주신 mobilenet v2 model 기반으로 다시 돌려보기


In [2]:
import torch
print(torch.__version__)

import cv2
from mobilenetv2 import mobilenet_v2

/home/sldev1/anaconda3/envs/dms2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


In [3]:
emo_model = mobilenet_v2(in_channels=3, num_classes=8) 
model_path = './mobilenet_v2_affectnet_2023_01_12_07.37'

state = torch.load(model_path, map_location='cpu')
emo_model.load_state_dict(state['net'])



<All keys matched successfully>

In [15]:
img = '/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png'

cap = cv2.VideoCapture(img)
ret, frame = cap.read()

    
# rgbframe = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# results = face_detection.process(rgbframe)



image_arr = cv2.resize(frame,(256,256))
print(image_arr.shape)
image_arr = image_arr.reshape(1,3,256,256)
print(image_arr.shape)
print(image_arr)
image_arr = image_arr[...,::-1].astype("float")
print(image_arr)
image_arr = image_arr/255.
print(type(image_arr))

# print(type(image_arr))
# image_tensor = torch.Tensor(image_arr)
# print(type(image_tensor))

# out = emo_model(image_tensor)
# expr = out['expression']
# expr = np.argmax(expr.cpu().detach().numpy(), axis=1)
# # expr = np.argmax(expr.cpu().numpy(), axis=1)
# print(expr) # type = numpy

(256, 256, 3)
(1, 3, 256, 256)
[[[[ 44  28  15 ... 255 250 255]
   [255 250 255 ... 199 247 225]
   [200 246 225 ... 239 217 184]
   ...
   [250 237 254 ... 194 236 218]
   [195 235 217 ... 231 211 180]
   [ 13   5   0 ... 255 254 255]]

  [[255 254 255 ... 199 243 226]
   [201 243 226 ... 231 211 180]
   [ 13   5   0 ... 255 254 254]
   ...
   [126 153 137 ... 196 180 155]
   [  0   0   0 ... 190 165 211]
   [190 164 204 ...  41 192 173]]

  [[141 176 159 ... 195 180 155]
   [  1   0   0 ... 190 165 211]
   [190 165 208 ...  70 190 175]
   ...
   [  1   0   0 ...  21  18  40]
   [ 39  36  69 ...  17  31  28]
   [ 21  36  34 ...  39  31  26]]]]
[[[[255. 250. 255. ...  15.  28.  44.]
   [225. 247. 199. ... 255. 250. 255.]
   [184. 217. 239. ... 225. 246. 200.]
   ...
   [218. 236. 194. ... 254. 237. 250.]
   [180. 211. 231. ... 217. 235. 195.]
   [255. 254. 255. ...   0.   5.  13.]]

  [[226. 243. 199. ... 255. 254. 255.]
   [180. 211. 231. ... 226. 243. 201.]
   [254. 254. 255. ...   0

In [16]:
image_arr = image_arr[...,::-1]
print(image_arr)

[[[[0.17254902 0.10980392 0.05882353 ... 1.         0.98039216
    1.        ]
   [1.         0.98039216 1.         ... 0.78039216 0.96862745
    0.88235294]
   [0.78431373 0.96470588 0.88235294 ... 0.9372549  0.85098039
    0.72156863]
   ...
   [0.98039216 0.92941176 0.99607843 ... 0.76078431 0.9254902
    0.85490196]
   [0.76470588 0.92156863 0.85098039 ... 0.90588235 0.82745098
    0.70588235]
   [0.05098039 0.01960784 0.         ... 1.         0.99607843
    1.        ]]

  [[1.         0.99607843 1.         ... 0.78039216 0.95294118
    0.88627451]
   [0.78823529 0.95294118 0.88627451 ... 0.90588235 0.82745098
    0.70588235]
   [0.05098039 0.01960784 0.         ... 1.         0.99607843
    0.99607843]
   ...
   [0.49411765 0.6        0.5372549  ... 0.76862745 0.70588235
    0.60784314]
   [0.         0.         0.         ... 0.74509804 0.64705882
    0.82745098]
   [0.74509804 0.64313725 0.8        ... 0.16078431 0.75294118
    0.67843137]]

  [[0.55294118 0.69019608 0.6235294

## 수정코드

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from emotion_torch_mobilenetV2 import Emotion
import mediapipe as mp
import csv
import glob
import re
from tqdm import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-hs_1wiee because the default path (/home/sldev1/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2023-01-19 16:07:14.912295: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sldev1/anaconda3/envs/dms2/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.0/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-01-19 16:07:14.912318: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/sldev1/anaconda3/envs/dms2/lib/python3.7

In [2]:
dirpath = "/media/sldev1/T7/DMS2/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.mp4')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))
filepath = filepath[7:9]
print(filepath)
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

['/media/sldev1/T7/DMS2/08. 정승아_1214.mp4', '/media/sldev1/T7/DMS2/09. 류채원_1214.mp4']


In [ ]:
for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_MobilenetV2/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
                rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(rgbframe)
                rgbframe.flags.writeable = True
                rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = rgbframe.shape[0]*bbox.ymin
                    bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
                    left = rgbframe.shape[1]*bbox.xmin
                    right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

                    pre_emo = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
#                     cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
#                     cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)


                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_MobilenetV2/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()
                
                if cap.get(1) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
            cap.release()
            cv2.destroyAllWindows()


# 1258 frame에서 멈추는 중

  0%|                                                                                           | 0/2 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


/media/sldev1/T7/DMS2/08. 정승아_1214.mp4


## 01/13(금) 오류

In [ ]:
  6%|████▌                                                                      | 3/50 [2:52:17<45:43:23, 3502.20s/it]
/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4
[h264 @ 0x559e98b11500] Invalid NAL unit size (7082 > 1541).
[h264 @ 0x559e98b11500] Error splitting the input into NAL units.
  6%|████▌                                                                      | 3/50 [2:54:15<45:30:00, 3485.13s/it]
---------------------------------------------------------------------------
error                                     Traceback (most recent call last)
/tmp/ipykernel_17891/1244803589.py in <module>
     12             while(cap.isOpened()):
     13                 ret, frame = cap.read()
---> 14                 rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
     15                 results = face_detection.process(rgbframe)
     16                 rgbframe.flags.writeable = True

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
                
# Frame 2566에서 에러뜸.
# 다시 하니까 또 됨. 머노?

## 01/18 (수) csv file editing

In [1]:
pwd

'/home/sldev1/github/TIL/Internship/DMS2_SMU'

In [12]:
import glob
import re
import csv

In [13]:
dirpath = "./FER_Result_SL/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.csv')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[0]))
filepath = filepath[0:2]
print(filepath)


['./FER_Result_SL/01. 김형민_1201.csv', './FER_Result_SL/02. 김두용_1205.csv']


In [15]:
v = open(filepath[0])
r = csv.reader(v)
row0 = r.next()
row0.append('Valence')
print(row0)

AttributeError: '_csv.reader' object has no attribute 'next'